In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import cv2

from dataset import ImageNet
from model import DarkNet
from trainer import Trainer

class config():
    def __init__(self):
        self.batch_size = 32
        self.lr = 0.001
        self.device='cuda'

        self.img_size = 448
        self.num_classes = 30

        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        self.test_transform = A.Compose([
            A.Normalize(mean=mean, std=std, max_pixel_value=255),
            A.LongestMaxSize(max_size=self.img_size),
            A.PadIfNeeded(
                min_height=self.img_size,
                min_width=self.img_size,
                border_mode=cv2.BORDER_CONSTANT,
            ),
            ToTensorV2(),
        ])

        class_names = [
            'tench',
            'goldfish',
            'great white shark',
            'tiger shark',
            'hammerhead',
            'electric ray',
            'stingray',
            'cock',
            'hen',
            'ostrich',
            'brambling',
            'goldfinch',
            'house finch',
            'junco',
            'indigo bunting',
            'robin',
            'bulbul',
            'jay',
            'magpie',
            'chickadee',
            'water ouzel',
            'kite',
            'bald eagle',
            'vulture',
            'great grey owl',
            'European fire salamander',
            'common newt',
            'eft',
            'spotted salamander',
            'axolotl',
        ]

        self.print_interval = 50
        model_dir = './' 
        self.load_model_path = os.path.join(model_dir, 'result/YOLOv1_pretrained_30epoch.pt')

        dataset_dir = r'D:\AI\Dataset\ImageNet'
        self.img_dir = os.path.join(dataset_dir, 'images')
        self.train_csv = os.path.join(dataset_dir, 'train.csv')
        self.valid_csv = os.path.join(dataset_dir, 'valid.csv')
        # self.train_csv = os.path.join(dataset_dir, '120_sample.csv')
        # self.valid_csv = os.path.join(dataset_dir, '120_sample.csv')

        self.csv_path = './result/YOLOv1_pretrained_result.csv'

config = config()

valid_dataset = ImageNet(
    config.valid_csv,
    config.img_dir,
    config.test_transform,
)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True,
)

model = DarkNet(config.num_classes).to(config.device)
optimizer = optim.Adam(model.parameters(), config.lr)
crit = nn.CrossEntropyLoss(reduction="sum")

trainer = Trainer(model, optimizer, crit)

# darknet 모델 체크포인트 생성

In [2]:
# load_model_path = './result/YOLOv1_pretrained_best_model.pt'
# save_model_path = './result/pretrained_darknet.pt'

# checkpoint = torch.load(load_model_path, map_location=config.device)
# model.load_state_dict(checkpoint['model'])
# torch.save(model.darknet.state_dict(), save_model_path)